# Poker Dataset con clasificación multiclase y regularización

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import json, matplotlib
#s = json.load( open("styles/bmh_matplotlibrc.json") )
#matplotlib.rcParams.update(s)
from IPython.core.pylabtools import figsize
from tqdm import tqdm
figsize(11, 5)
colores = ["#348ABD", "#A60628","#06A628"]

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

In [3]:
import numpy as np

## Función de activación
### Logística

In [4]:
# Devuelve la función logística evaluada
# componente por componente
def logistica(z):
    return 1 / (1 + np.exp(-z))

In [5]:
## Función que, dado un arreglo de valores z
## calcula el valor de la derivada para cada entrada.

def derivadaLogistica(z):
    g = logistica(z)
    return g * (1 - g)

### Softmax

In [6]:
#z = [1.0, 2.0, 3.0, 4.0, 1.0, 2.0, 3.0]
def softmax(z):
    z_exp = [np.exp(i) for i in z]
    #print([round(i, 2) for i in z_exp])
    #[2.72, 7.39, 20.09, 54.6, 2.72, 7.39, 20.09]
    sum_z_exp = sum(z_exp)
    #print(round(sum_z_exp, 2))
    #114.98
    soft_m = [(i / sum_z_exp) for i in z_exp]
    #print(softmax)
    #[0.024, 0.064, 0.175, 0.475, 0.024, 0.064, 0.175]
    return np.array(soft_m)

## Función de Error

In [7]:
# input: (val_obtenido, val_esperado)
def entropia_cruzada(hip,y):
    return -y*np.log(hip) - (1-y)*np.log(1-hip)

def entropia_multiclase(hip,y):
    return -np.log(hip[y]) #+regularización

## Red neuronal
La red implementa encadenamiento hacia adelante (para evaluar) y hacia atrás (para entrenarse).

In [8]:
np.random.seed(10)

In [9]:
class POKER:
    def __init__(self,lamb = 0.3):
        
        self.lamb = lamb
        
        neg_pos1 = np.power(-1, np.random.randint(50, size=(6, 11))) #para hacerlos negativos o positivos
        neg_pos2 = np.power(-1, np.random.randint(50, size=(10, 7)))
        
        self.Theta_0 = np.multiply(np.random.random((6,11)), neg_pos1)
        self.Theta_1 = np.multiply(np.random.random((10,7)), neg_pos2)
    
    def feedForward(self, X, vector = None):
        """ Calcula las salidas, dados los datos de entrada. """
        if vector is None:
            Theta_0 = self.Theta_0
            Theta_1 = self.Theta_1
        else:
            Theta_0, Theta_1 = self.reconstructMatrices(vector)
        
        self.A0 = np.vstack((np.ones((1, X.shape[0])), X.T)) #entrada
        
        self.Z1 = np.dot(Theta_0, self.A0)#primera capa
        self.A1 = np.vstack((np.ones((1, self.Z1.shape[1])), logistica(self.Z1)))
        
        self.Z2 = np.dot(Theta_1, self.A1) #última capa
        self.A2 = softmax(self.Z2)
        
    
        
    def backPropagate(self, X, Y):
        """ Calcula el error y su gradiente,
        dados los pesos actuales de la red y los resultados
        esperados.
        """
        self.feedForward(X)
        
        m = X.shape[0]
        #Delta_2 = (self.A2.T - Y)
        A2T = np.copy(self.A2.T)
        for i in range(len(Y)):
            A2T[i][Y[i]] -= 1
        Delta_2 = A2T #Delta_2 = (self.A2.T - Y)
        
        n = X.shape[0]
        self.error = np.sum(entropia_multiclase(self.A2,Y.T)) / (2*n) + (np.sum(np.power(self.Theta_0,2)) + np.sum(np.power(self.Theta_1,2)))  * self.lamb/(2*n)
        
        #Delta_1 = Delta_2 * self.Theta_1[:,1:] *  derivadaLogistica(self.Z2) #paraxor
        Delta_1 = np.multiply(Delta_2, derivadaLogistica(self.Z2.T)).dot(self.Theta_1[:,1:]*self.lamb)
        
        self.Grad_1 = np.dot(Delta_2.T,self.A1.T) / m
        
        #Delta_0 = self.Theta_0[:,1:].T * Delta_1 * derivadaLogistica(self.Z1.T)
        self.Grad_0 = np.dot(Delta_1.T, self.A0.T) / m
        
    def calcError(self, X, Y, vector):
        """
        Calcula el error que se cometería utilizando los pesos en 'vector'.
        """
        self.feedForward(X, vector)
        n = X.shape[0]
        #funcion de predida = funcion de error + regularización
        error = np.sum(entropia_multiclase(self.A2,Y.T)) / (2*n) + (np.sum(np.power(self.Theta_0,2)) + np.sum(np.power(self.Theta_1)))  * self.lamb/(2*n)
        return error
    
    def vectorWeights(self):
        """
        Acomoda a todos los parámetros en las matrices de pesos, en un solo vector.
        """
        vector = np.vstack((self.Theta_0.reshape((self.Theta_0.size, 1)),
                          self.Theta_1.reshape((self.Theta_1.size, 1))))
        #print(self.Theta_0, self.Theta_1, vector)
        return vector
    
    def reconstructMatrices(self, vector):
        """
        Dado un vector, rearma matrices del tamaño de las matrices de pesos.
        """
        M0 = vector[0:self.Theta_0.size].reshape(self.Theta_0.shape)
        M1 = vector[self.Theta_0.size:].reshape(self.Theta_1.shape)
        return M0, M1
        
    def approxGradient(self, X, Y):
        """
        Aproxima el valor del gradiente alrededor de los pesos actuales,
        perturbando cada valor, uno por uno.
        """
        vector = self.vectorWeights().copy()
        approx = np.zeros(vector.shape)
        perturb = np.zeros(vector.shape)
        epsilon = 0.0001
        
        for i in range(len(vector)):
            perturb[i] = epsilon
            loss1 = self.calcError(X, Y, vector - perturb)
            loss2 = self.calcError(X, Y, vector + perturb)
            perturb[i] = 0
            approx[i] = (loss2 - loss1) / (2 * epsilon)
        return self.reconstructMatrices(approx)
        
    def gradientDescent(self, X, Y, alpha, ciclos=10, checkGradient = False):
        """ Evalúa y ajusta los pesos de la red,
        de acuerdo a los datos en X y los resultados
        esperados, en Y.
        """
        errores = np.zeros(ciclos)
        for i in tqdm(range(ciclos)):
            self.backPropagate(X, Y)
            Grad_1 = self.Grad_1
            Grad_0 = self.Grad_0
            if checkGradient:
                ApproxT0, ApproxT1 = self.approxGradient(X, Y)
                print("Grad 0 = ", Grad_0, "\nApprox = ", ApproxT0, "\nDiff = ", Grad_0 - ApproxT0,
                     "\nGrad 1 = ", Grad_1, "\nApprox = ", ApproxT1, "\nDiff = ", Grad_1 - ApproxT1)
            self.Theta_1 -= alpha * Grad_1
            self.Theta_0 -= alpha * Grad_0
            errores[i] = self.error
        if ciclos > 1:
            plt.plot(np.arange(ciclos), errores)
        
    def printOutput(self):
        print(self.A2.T)

## Cargando conjunto de entrenamiento

In [10]:
#xor.printOutput()
X,Y=[],[]
fh = open('poker-hand-testing.data')
for i in tqdm(range(1000)):
    row = list(map(int,fh.readline().strip().split(",")))
    X.append(row[:-1])
    Y.append(row[-1:])
    
X = np.array(X)
Y = np.array(Y)
fh.close()

poker = POKER(0.3) #se crea con lambda para la regularización
poker.feedForward(X)

100%|██████████| 1000/1000 [00:00<00:00, 36082.83it/s]


In [11]:
#print(poker.vectorWeights())
T0, T1 = poker.reconstructMatrices(poker.vectorWeights())
#print(T0,"\n\n\n", T1)

In [12]:
poker.gradientDescent(X, Y, 0.3, 1, checkGradient = False)

100%|██████████| 1/1 [00:00<00:00, 51.90it/s]


In [13]:
@interact_manual(ciclos = (50, 2000))
def trainXOR(ciclos):
    poker.gradientDescent(X, Y, 0.3, ciclos)

interactive(children=(IntSlider(value=1025, description='ciclos', max=2000, min=50), Button(description='Run I…

In [14]:
poker.feedForward(X)
poker.printOutput()
print("Theta_0 = ", poker.Theta_0, "\nTheta_1", poker.Theta_1)
print(poker.error)

[[0.13395657 0.09183451 0.26528172 ... 0.0885414  0.06988882 0.04984132]
 [0.34920979 0.06049258 0.17988586 ... 0.09702005 0.02993837 0.06355415]
 [0.1586271  0.10089314 0.22769679 ... 0.10305498 0.05692578 0.05172921]
 ...
 [0.18219795 0.04525798 0.28176467 ... 0.09338815 0.06263516 0.05020861]
 [0.15077101 0.0421143  0.29791893 ... 0.09845105 0.07277103 0.04817423]
 [0.14216999 0.10258269 0.24050582 ... 0.08755199 0.06015466 0.04562187]]
Theta_0 =  [[-0.27606641  0.52361344  0.00996022  0.34993982  0.0511285   0.68247044
  -0.32604638 -0.72936672  0.18922625 -0.36992982 -0.81487628]
 [ 0.95707683 -0.48569042  0.1072291   0.34081591  0.83692054  0.46624498
   0.09353867  0.85520724 -0.33464041 -0.60576417  0.11436917]
 [-0.53492035 -0.69830158  0.9270135   0.16924342  0.16764957 -0.46757434
   0.03453198 -0.57188596 -0.14260037 -0.18023277 -0.77969072]
 [ 0.05654904  0.48598337  0.21210816  0.31430029 -0.03719136  0.28222884
  -1.01797232 -0.25087075  0.69550165 -0.97395519  0.7805038

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../P1-HodgkinHuxley/styles/custom.css", "r").read() #or edit path to custom.css
    return HTML(styles)
css_styling()